# Jaeger-query with trace DSL

This notebook loads a trace from Jaeger query service via gRPC and runs analysis on it.

## Install library to the local maven repository
This step is only needed if trace DSL source code has been modified.
Open terminal in Jupyter and run the following command:
```
cd work && ./mvnw clean install -DskipTests
```

In [1]:
%%loadFromPOM
<dependency>
  <groupId>com.google.protobuf</groupId>
  <artifactId>protobuf-java</artifactId>
  <version>3.11.4</version>
</dependency>
<dependency>
  <groupId>com.google.protobuf</groupId>
  <artifactId>protobuf-java-util</artifactId>
  <version>3.11.4</version>
</dependency>
<dependency>
  <groupId>io.grpc</groupId>
  <artifactId>grpc-protobuf</artifactId>
  <version>1.28.0</version>
</dependency>
<dependency>
  <groupId>io.grpc</groupId>
  <artifactId>grpc-netty</artifactId>
  <version>1.28.0</version>
</dependency>
<dependency>
  <groupId>io.grpc</groupId>
  <artifactId>grpc-stub</artifactId>
  <version>1.28.0</version>
</dependency>

<dependency>
    <groupId>io.prometheus</groupId>
    <artifactId>simpleclient_httpserver</artifactId>
    <version>0.7.0</version>
</dependency>
<dependency>
    <groupId>org.apache.tinkerpop</groupId>
    <artifactId>tinkergraph-gremlin</artifactId>
    <version>3.4.3</version>
</dependency>
<dependency>
    <groupId>log4j</groupId>
    <artifactId>log4j</artifactId>
    <version>1.2.17</version>
</dependency>


In [2]:
%maven io.jaegertracing:jaeger-tracedsl:0.4.0
%maven io.jaegertracing:jaeger-proto:0.4.0

System.out.println(io.jaegertracing.analytics.gremlin.Keys.class);

class io.jaegertracing.analytics.gremlin.Keys


## Run analysis
It calculates maximum trace height and network latencies. It connects to Jaeger query service via gRPC.

In [10]:
import com.google.protobuf.ByteString;
import io.grpc.ManagedChannel;
import io.grpc.ManagedChannelBuilder;
import io.jaegertracing.analytics.*;
import io.jaegertracing.analytics.NetworkLatency.Name;
import io.jaegertracing.analytics.gremlin.GraphCreator;
import io.jaegertracing.analytics.model.Converter;
import io.jaegertracing.analytics.model.Trace;
import io.jaegertracing.api_v2.Model.Span;
import io.jaegertracing.api_v2.Query.GetTraceRequest;
import io.jaegertracing.api_v2.Query.SpansResponseChunk;
import io.jaegertracing.api_v2.QueryServiceGrpc;
import io.jaegertracing.api_v2.Query.FindTracesRequest;
import io.jaegertracing.api_v2.QueryServiceGrpc.QueryServiceBlockingStub;
import io.jaegertracing.api_v2.Query.TraceQueryParameters;
import io.opentracing.tag.Tags;
import java.util.ArrayList;
import java.util.Iterator;
import java.util.List;
import java.util.Map;
import java.util.Set;
import org.apache.tinkerpop.gremlin.structure.Graph;

String queryHostPort = "192.168.0.31:16686";

ManagedChannel channel;
channel = ManagedChannelBuilder.forTarget(queryHostPort).usePlaintext().build();
QueryServiceBlockingStub queryService = QueryServiceGrpc.newBlockingStub(channel);

TraceQueryParameters query = TraceQueryParameters.newBuilder().setServiceName("frontend").build();
Iterator<SpansResponseChunk> traceProto = queryService.findTraces(
    FindTracesRequest.newBuilder().setQuery(query).build());

List<Span> spans = new ArrayList<>();
while (traceProto.hasNext()) {
  spans.addAll(traceProto.next().getSpansList());
}
Trace trace = Converter.toModel(spans);
Graph graph = GraphCreator.create(trace);

Set<io.jaegertracing.analytics.model.Span> errorSpans = NumberOfErrors.calculate(graph);
Map<String, Integer> errorTypeAndCount = new LinkedHashMap<>();
for (io.jaegertracing.analytics.model.Span errorSpan: errorSpans) {
  for (io.jaegertracing.analytics.model.Span.Log log: errorSpan.logs) {
    String err = log.fields.get(Tags.ERROR.getKey());
    if (err != null) {
      Integer count = errorTypeAndCount.get(err);
      if (count == null) {
        count = 0;
      }
      errorTypeAndCount.put(err, ++count);
    }
  }
}
System.out.printf("Error and count: %s\n", errorTypeAndCount);

int height = TraceHeight.calculate(graph);
Map<Name, Set<Double>> networkLatencies = NetworkLatency.calculate(graph);
System.out.printf("Trace height = %d\n", height);
System.out.printf("Network latencies = %s\n", networkLatencies);


Error and count: {redis timeout=5}
Trace height = 4
Network latencies = {Name{client='frontend', server='route'}=[2.47E-4, 2.9E-4, 4.08E-4, 3.03E-4, 2.22E-4, 1.52E-4, 1.83E-4, 3.82E-4, 1.31E-4, 1.98E-4, 2.78E-4, 1.68E-4, 2.33E-4, 1.23E-4, 2.09E-4, 1.4E-4, 3.6E-4, 1.25E-4, 2.93E-4, 4.58E-4], Name{client='frontend', server='customer'}=[3.91E-4, 1.8E-4], Name{client='frontend', server='driver'}=[5.14E-4, 4.78E-4]}


java.io.PrintStream@1fdec01f

## Write your own data analysis with Trace DSL and Apache Gremlin

In [31]:
// Are two spans connected?
TraceTraversal<Vertex, Vertex> traversal = graph.traversal(TraceTraversalSource.class)
        .hasName("HTTP GET: /customer")
        .repeat(__.out())
        .until(__.hasName("SQL SELECT"));

traversal.forEachRemaining(v -> Util.printVertex(v));
